In [1]:
import sys
sys.path.append('../')

In [2]:
import numpy as np
from ortools.sat.python import cp_model

from vrp_study.data_model import Tariff, Cargo, Node
from vrp_study.data_model import TariffCost
from vrp_study.routing_manager import PDRoutingManager
from vrp_study.ortools_routing_model.routing_model import find_optimal_paths

In [3]:
benchmark_type = 'pdp_100'
name = 'lc101.txt'

In [4]:
from typing import Optional

tariff = None
cargos: list[Cargo] = []
depo: Optional[Node] = None

In [5]:
id2info = {}
p2coordinates = {}
with open(f'../data/Li & Lim benchmark/{benchmark_type}/{name}', 'r') as file:
    for i, line in enumerate(file):

        print(line)
        line = line.split('\t')
        if i == 0:
            tariff = Tariff(
                id='car',
                capacity=int(line[1]),
                max_count=int(line[0]),
                cost_per_distance=[TariffCost(
                    min_dst_km=0,
                    max_dst_km=10000,
                    cost_per_km=1,
                    fixed_cost=0
                )]
            )
        else:
            c_id = int(line[0])
            x = int(line[1])
            y = int(line[2])

            mass = int(line[3])

            et = int(line[4])
            lt = int(line[5])
            st = int(line[6])

            pick_up = int(line[7])
            delivery = int(line[8])
            if pick_up == delivery:
                # print(12)
                depo = Node(
                    id=0,
                    cargo_id=c_id,
                    capacity=0,
                    service_time=0,
                    start_time=0,
                    end_time=lt,
                    coordinates=(x, y)
                )
                continue
            if pick_up == 0:
                if c_id not in id2info:
                    id2info[c_id] = {}
                id2info[c_id][0] = (x, y, mass, et, lt, st, c_id, delivery)
            else:
                delivery = c_id
                c_id = pick_up
                if c_id not in id2info:
                    id2info[c_id] = {}
                id2info[c_id][1] = (x, y, mass, et, lt, st, pick_up, delivery)


25	200	1

0	40	50	0	0	1236	0	0	0

1	45	68	-10	912	967	90	11	0

2	45	70	-20	825	870	90	6	0

3	42	66	10	65	146	90	0	75

4	42	68	-10	727	782	90	9	0

5	42	65	10	15	67	90	0	7

6	40	69	20	621	702	90	0	2

7	40	66	-10	170	225	90	5	0

8	38	68	20	255	324	90	0	10

9	38	70	10	534	605	90	0	4

10	35	66	-20	357	410	90	8	0

11	35	69	10	448	505	90	0	1

12	25	85	-20	652	721	90	18	0

13	22	75	30	30	92	90	0	17

14	22	85	-40	567	620	90	16	0

15	20	80	-10	384	429	90	19	0

16	20	85	40	475	528	90	0	14

17	18	75	-30	99	148	90	13	0

18	15	75	20	179	254	90	0	12

19	15	80	10	278	345	90	0	15

20	30	50	10	10	73	90	0	24

21	30	52	-10	914	965	90	30	0

22	28	52	-20	812	883	90	28	0

23	28	55	10	732	777	0	0	103

24	25	50	-10	65	144	90	20	0

25	25	52	40	169	224	90	0	27

26	25	55	-10	622	701	90	29	0

27	23	52	-40	261	316	90	25	0

28	23	55	20	546	593	90	0	22

29	20	50	10	358	405	90	0	26

30	20	55	10	449	504	90	0	21

31	10	35	-30	200	237	90	32	0

32	10	40	30	31	100	90	0	31

33	8	40	40	87	158	90	0	37

34	8	45	-30	751	816	90	

In [6]:
depo

Node(id=0, cargo_id=0, capacity=0, start_time=0, end_time=1236, service_time=0, coordinates=(40, 50))

In [7]:

for k, v in id2info.items():
    cargos.append(
        Cargo(
            id=k,
            nodes=[
                Node(
                    cargo_id=k,
                    id=v[i][6] if i == 0 else v[i][7],
                    capacity=v[i][2],
                    service_time=v[i][5],
                    start_time=v[i][3],
                    end_time=v[i][4],
                    coordinates=(v[i][0], v[i][1])
                )
                for i in range(2)
            ]
        )
    )

In [8]:
cargos[0]

Cargo(id=11, nodes=[Node(id=11, cargo_id=11, capacity=10, start_time=448, end_time=505, service_time=90, coordinates=(35, 69)), Node(id=1, cargo_id=11, capacity=-10, start_time=912, end_time=967, service_time=90, coordinates=(45, 68))])

In [9]:
tariff

Tariff(id='car', capacity=200, cost_per_distance=[TariffCost(min_dst_km=0, max_dst_km=10000, cost_per_km=1, fixed_cost=0)], max_count=25)

In [10]:
p2coordinates.update({
    crg.nodes[i].id: crg.nodes[i].coordinates for crg in cargos for i in range(2)
})
p2coordinates[depo.id] = depo.coordinates
distance_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in
                   p2coordinates.items() for
                   v, dv in p2coordinates.items()}
time_matrix = {(u, v): np.sqrt((du[0] - dv[0]) ** 2 + (du[1] - dv[1]) ** 2) for u, du in p2coordinates.items() for
               v, dv in p2coordinates.items()}

In [11]:

routing_manager = PDRoutingManager()

routing_manager.add_cargos(cargos)
routing_manager.add_tariff(tariff)

routing_manager.add_depo(depo)

routing_manager.distance_matrix = distance_matrix
routing_manager.time_matrix = time_matrix

routing_manager.max_time_minutes = 1

routing_manager.build()

In [ ]:
sol = find_optimal_paths(routing_manager)

2025-06-18 10:43:11,052 - routing_model [INFO] - problem size: 107
2025-06-18 10:43:11,053 - routing_model [INFO] - Начало создания модели
2025-06-18 10:43:11,056 - routing_model [INFO] - Добавление размерности для расстояния
2025-06-18 10:43:11,057 - routing_model [INFO] - Добавление размерности для расстояния
2025-06-18 10:43:11,058 - routing_model [INFO] - Добавление ограничения для порядка доставки
2025-06-18 10:43:11,059 - routing_model [INFO] - Добавление стоимостей машин
2025-06-18 10:43:11,060 - routing_model [INFO] - add time
2025-06-18 10:43:11,062 - routing_model [INFO] - Добавление ограничений для массы
2025-06-18 10:43:11,070 - routing_model [INFO] - Начало решения
2025-06-18 10:43:11,311 - routing_model [INFO] - find new solution: 82873, best solution: 82873
2025-06-18 10:43:12,294 - routing_model [INFO] - find new solution: 85686, best solution: 82873
2025-06-18 10:43:12,459 - routing_model [INFO] - find new solution: 82873, best solution: 82873
2025-06-18 10:43:12,700 -

In [ ]:
#  best_score: 107
# 2025-06-02 14:45:05,380 - routing_model [INFO] - best_score: 46228.27

In [ ]:
# est_score: 8
# 2025-06-02 14:25:30,179 - routing_model [INFO] - best_score: 1979.21

In [ ]:
cargos[0]

In [ ]:
total_l = 0
for s in sol[0]:
    if len(s) > 0:
        # print(s)
        l = sum(routing_manager.get_distance(s[i], s[i + 1]) for i in range(len(s) - 1))
        # print(s, l)
        total_l += l
total_l

In [ ]:
for crg in cargos:
    in1 = routing_manager._node_to_inner_node[crg.nodes[0]]
    in2 = routing_manager._node_to_inner_node[crg.nodes[1]]

    start = None
    end = None
    in_time = None
    out_time = None

    for i, s in enumerate(sol[0]):
        s_time = sol[1][i]
        if len(s) == 0:
            continue

        for order, i in enumerate(s):
            if i == in1.id:
                start = i
                in_time = s_time[order]
            if i == in2.id:
                end = i
                out_time = s_time[order]

    assert start is not None and end is not None and start < end, (start, end)

    if not (crg.nodes[0].start_time <= in_time[0] and crg.nodes[0].end_time >= in_time[1]):
        print(in_time, crg.nodes[0].start_time, crg.nodes[0].end_time)
    if not (crg.nodes[1].start_time <= out_time[0] and crg.nodes[1].end_time >= out_time[1]):
        print(out_time, crg.nodes[0].start_time, crg.nodes[0].end_time)